# 📊 Corporate Financial Health Analyzer
## Team 10 - Altman Z-Score Analysis

This notebook demonstrates the step-by-step process of analyzing corporate financial health using the Altman Z-Score model.

---
## Step 1: Import Required Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries imported successfully")

---
## Step 2: Define Industry Benchmarks

We'll use these industry categories for comparison analysis.

In [ ]:
INDUSTRIES = {
    'Manufacturing': ['F', 'GM', 'CAT', 'DE', 'BA', 'GE', 'MMM', 'HON'],
    'Retail': ['WMT', 'TGT', 'COST', 'HD', 'LOW', 'AMZN', 'EBAY'],
    'Technology': ['AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'AMD', 'INTC', 'ORCL'],
    'Healthcare': ['JNJ', 'UNH', 'PFE', 'ABBV', 'TMO', 'ABT', 'DHR', 'BMY'],
    'Food & Beverage': ['KO', 'PEP', 'MCD', 'SBUX', 'KHC', 'GIS', 'K', 'HSY'],
    'Transportation': ['UPS', 'FDX', 'UAL', 'DAL', 'AAL', 'LUV', 'NSC', 'UNP'],
    'Energy': ['XOM', 'CVX', 'COP', 'SLB', 'EOG', 'MPC', 'PSX', 'VLO'],
    'Finance': ['JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'BLK', 'SCHW'],
    'Automotive': ['TSLA', 'F', 'GM', 'TM', 'HMC', 'STLA', 'RIVN']
}

print(f"✓ Defined {len(INDUSTRIES)} industry categories")
print(f"Total companies: {sum(len(v) for v in INDUSTRIES.values())}")

---
## Step 3: Fetch Financial Data from Yahoo Finance

In [ ]:
def fetch_financials(ticker):
    """
    Fetch quarterly financial data from Yahoo Finance.
    
    Parameters:
        ticker (str): Stock ticker symbol
    
    Returns:
        dict: Financial data or None if fetch fails
    """
    try:
        company = yf.Ticker(ticker)
        bs = company.quarterly_balance_sheet.iloc[:, 0]
        inc = company.quarterly_financials.iloc[:, 0]
        
        data = {
            'total_assets': bs.get('Total Assets', 0),
            'current_assets': bs.get('Current Assets', 0),
            'current_liabilities': bs.get('Current Liabilities', 0),
            'retained_earnings': bs.get('Retained Earnings', 0),
            'ebit': inc.get('EBIT', inc.get('Operating Income', 0)),
            'total_liabilities': bs.get('Total Liabilities Net Minority Interest', 
                                       bs.get('Total Liabilities', 0)),
            'market_value_equity': bs.get('Common Stock Equity', 
                                         bs.get('Stockholders Equity', 0)),
            'total_revenue': inc.get('Total Revenue', 0)
        }
        
        if data['total_assets'] == 0 or data['total_revenue'] == 0:
            return None
        return data
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return None

print("✓ Financial data fetching function defined")

---
## Step 4: Calculate Altman Z-Score

### Formula:
**Z = 1.2×X1 + 1.4×X2 + 3.3×X3 + 0.6×X4 + 1.0×X5**

Where:
- **X1** = Working Capital / Total Assets
- **X2** = Retained Earnings / Total Assets
- **X3** = EBIT / Total Assets
- **X4** = Market Value of Equity / Total Liabilities
- **X5** = Sales / Total Assets

In [ ]:
def calculate_z_score(data):
    """
    Calculate Altman Z-Score from financial data.
    
    Parameters:
        data (dict): Financial data dictionary
    
    Returns:
        pd.Series: Calculated ratios and Z-Score
    """
    df = pd.DataFrame([data])
    
    # Calculate working capital
    wc = df['current_assets'] - df['current_liabilities']
    
    # Calculate component ratios
    df['x1'] = wc / df['total_assets']
    df['x2'] = df['retained_earnings'] / df['total_assets']
    df['x3'] = df['ebit'] / df['total_assets']
    df['x4'] = df['market_value_equity'] / df['total_liabilities'].replace(0, 1)
    df['x5'] = df['total_revenue'] / df['total_assets']
    
    # Calculate Z-Score
    df['z_score'] = (1.2 * df['x1'] + 
                     1.4 * df['x2'] + 
                     3.3 * df['x3'] + 
                     0.6 * df['x4'] + 
                     1.0 * df['x5'])
    
    return df.iloc[0]

print("✓ Z-Score calculation function defined")

---
## Step 5: Define Risk Classification

### Z-Score Interpretation:
- 🟢 **Safe Zone**: Z > 2.99 (Low bankruptcy risk)
- 🟡 **Gray Zone**: 1.81 ≤ Z ≤ 2.99 (Moderate risk)
- 🔴 **Distress Zone**: Z < 1.81 (High bankruptcy risk)

In [ ]:
def get_risk_zone(z_score):
    """
    Classify financial health based on Altman Z-Score.
    
    Parameters:
        z_score (float): Calculated Z-Score
    
    Returns:
        tuple: (zone_name, color, emoji)
    """
    if z_score > 2.99:
        return 'Safe Zone', '#28a745', '🟢'
    elif z_score >= 1.81:
        return 'Gray Zone', '#ffc107', '🟡'
    return 'Distress Zone', '#dc3545', '🔴'

print("✓ Risk classification function defined")

---
## Step 6: Example Analysis - Single Company



In [ ]:
# Fetch data for Apple
ticker = 'AAPL'
print(f"Fetching financial data for {ticker}...")

data = fetch_financials(ticker)

if data:
    print(f"\n✓ Successfully fetched data for {ticker}")
    
    # Display raw financial data
    financial_df = pd.DataFrame([data]).T
    financial_df.columns = ['Value ($)']
    financial_df['Value ($)'] = financial_df['Value ($)'].apply(lambda x: f"{x:,.0f}")
    print("\n📊 Financial Data:")
    display(financial_df)
else:
    print(f"❌ Failed to fetch data for {ticker}")

---
## Step 7: Calculate Z-Score for the Company

In [ ]:
if data:
    # Calculate Z-Score
    result = calculate_z_score(data)
    z_score = result['z_score']
    zone, color, emoji = get_risk_zone(z_score)
    
    print(f"\n{'='*60}")
    print(f"  {ticker} - ALTMAN Z-SCORE ANALYSIS")
    print(f"{'='*60}")
    print(f"\n  Z-Score: {z_score:.2f}")
    print(f"  Classification: {emoji} {zone}")
    print(f"\n{'='*60}")
    
    # Component breakdown
    components_df = pd.DataFrame({
        'Component': [
            'X1: Working Capital / Total Assets',
            'X2: Retained Earnings / Total Assets',
            'X3: EBIT / Total Assets',
            'X4: Market Value / Total Liabilities',
            'X5: Sales / Total Assets'
        ],
        'Ratio': [result['x1'], result['x2'], result['x3'], result['x4'], result['x5']],
        'Weight': [1.2, 1.4, 3.3, 0.6, 1.0],
        'Contribution': [
            1.2*result['x1'], 
            1.4*result['x2'], 
            3.3*result['x3'],
            0.6*result['x4'], 
            1.0*result['x5']
        ]
    })
    
    print("\n📊 Component Breakdown:")
    display(components_df.style.format({
        'Ratio': '{:.4f}',
        'Weight': '{:.1f}',
        'Contribution': '{:.4f}'
    }))

---
## Step 8: Visualize Component Contributions

In [ ]:
if data:
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Chart 1: Z-Score Gauge
    ax1.barh([ticker], [z_score], color=color, alpha=0.7, edgecolor='black', linewidth=2)
    ax1.axvline(2.99, color='green', linestyle='--', label='Safe Zone (>2.99)', linewidth=2)
    ax1.axvline(1.81, color='orange', linestyle='--', label='Gray Zone (1.81-2.99)', linewidth=2)
    ax1.set_xlabel('Z-Score', fontsize=12, fontweight='bold')
    ax1.set_title(f'{ticker} - Altman Z-Score: {z_score:.2f}', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(axis='x', alpha=0.3)
    
    # Chart 2: Component Contributions
    components = ['X1\nWorking\nCapital', 'X2\nRetained\nEarnings', 
                  'X3\nEBIT', 'X4\nMarket\nValue', 'X5\nSales']
    contributions = [1.2*result['x1'], 1.4*result['x2'], 3.3*result['x3'], 
                     0.6*result['x4'], 1.0*result['x5']]
    
    bars = ax2.bar(components, contributions, color='steelblue', alpha=0.7, 
                   edgecolor='black', linewidth=2)
    ax2.set_ylabel('Contribution to Z-Score', fontsize=12, fontweight='bold')
    ax2.set_title('Component Contributions', fontsize=14, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Visualization complete for {ticker}")

---
## Step 9: Industry Benchmark Analysis



In [ ]:
def get_industry_benchmark(industry, max_companies=8):
    """
    Fetch live benchmark data from industry companies.
    
    Parameters:
        industry (str): Industry name
        max_companies (int): Maximum number of companies to analyze
    
    Returns:
        dict: Benchmark statistics
    """
    tickers = INDUSTRIES.get(industry, INDUSTRIES['Technology'])[:max_companies]
    
    z_scores = []
    successful_tickers = []
    
    print(f"\nFetching data for {industry} industry...")
    
    for ticker in tickers:
        print(f"  Fetching {ticker}...", end=" ")
        data = fetch_financials(ticker)
        if data:
            result = calculate_z_score(data)
            if not np.isnan(result['z_score']) and not np.isinf(result['z_score']):
                z_scores.append(result['z_score'])
                successful_tickers.append(ticker)
                print(f"✓ Z-Score: {result['z_score']:.2f}")
            else:
                print("✗ Invalid Z-Score")
        else:
            print("✗ Failed")
    
    print(f"\n✓ Successfully fetched {len(z_scores)} companies")
    
    return {
        'industry': industry,
        'count': len(z_scores),
        'tickers': successful_tickers,
        'avg': np.mean(z_scores),
        'median': np.median(z_scores),
        'top_25': np.percentile(z_scores, 75),
        'bottom_25': np.percentile(z_scores, 25),
        'scores': z_scores
    }

# Fetch benchmark for Technology industry
benchmark = get_industry_benchmark('Technology', max_companies=6)

---
## Step 10: Display Benchmark Statistics

In [ ]:
print(f"\n{'='*60}")
print(f"  {benchmark['industry'].upper()} INDUSTRY BENCHMARK")
print(f"{'='*60}")
print(f"\n  Companies Analyzed: {benchmark['count']}")
print(f"  Tickers: {', '.join(benchmark['tickers'])}")
print(f"\n  Average Z-Score: {benchmark['avg']:.2f}")
print(f"  Median Z-Score: {benchmark['median']:.2f}")
print(f"  Top 25% (75th percentile): {benchmark['top_25']:.2f}")
print(f"  Bottom 25% (25th percentile): {benchmark['bottom_25']:.2f}")
print(f"\n{'='*60}")

# Create benchmark comparison DataFrame
benchmark_df = pd.DataFrame({
    'Ticker': benchmark['tickers'],
    'Z-Score': benchmark['scores']
}).sort_values('Z-Score', ascending=False)

print("\n📊 Industry Z-Score Rankings:")
display(benchmark_df.style.format({'Z-Score': '{:.2f}'}))

---
## Step 11: Comparative Visualization

In [ ]:
if data and benchmark:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Chart 1: Company vs Industry Comparison
    companies = [ticker, 'Industry\nAvg', 'Top 25%', 'Bottom 25%']
    scores = [z_score, benchmark['avg'], benchmark['top_25'], benchmark['bottom_25']]
    colors_comp = [color, '#6c757d', '#28a745', '#dc3545']
    
    bars = ax1.bar(companies, scores, color=colors_comp, alpha=0.7, 
                   edgecolor='black', linewidth=2)
    ax1.set_ylabel('Z-Score', fontsize=12, fontweight='bold')
    ax1.set_title(f'{ticker} vs {benchmark["industry"]} Industry', 
                  fontsize=14, fontweight='bold')
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', fontweight='bold')
    
    # Chart 2: Industry Distribution
    ax2.hist(benchmark['scores'], bins=10, color='lightblue', 
             edgecolor='black', alpha=0.7)
    ax2.axvline(z_score, color=color, linestyle='--', linewidth=3, 
                label=f'{ticker}: {z_score:.2f}')
    ax2.axvline(benchmark['avg'], color='gray', linestyle=':', linewidth=2, 
                label=f'Industry Avg: {benchmark["avg"]:.2f}')
    ax2.set_xlabel('Z-Score', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax2.set_title('Industry Z-Score Distribution', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Comparative analysis complete")

---
## Step 12: Final Recommendations

In [ ]:
if data:
    print(f"\n{'='*60}")
    print(f"  RECOMMENDATIONS FOR {ticker}")
    print(f"{'='*60}\n")
    
    if z_score > 2.99:
        print("  ✓ EXCELLENT FINANCIAL HEALTH")
        print("  ━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print("  • Company is in the Safe Zone")
        print("  • Low bankruptcy risk")
        print("  • Focus on strategic growth opportunities")
        print("  • Maintain strong financial position")
    elif z_score >= 1.81:
        print("  ⚠ MODERATE RISK DETECTED")
        print("  ━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print("  • Company is in the Gray Zone")
        print("  • Monitor liquidity closely")
        print("  • Improve profitability metrics")
        print("  • Consider cost reduction strategies")
    else:
        print("  ✗ HIGH DISTRESS RISK")
        print("  ━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print("  • Company is in the Distress Zone")
        print("  • Immediate action required")
        print("  • Restructure debt obligations")
        print("  • Improve operational efficiency")
        print("  • Consider asset liquidation if necessary")
    
    # Industry position
    if z_score >= benchmark['top_25']:
        position = "Top 25% of industry"
    elif z_score >= benchmark['median']:
        position = "Above industry median"
    elif z_score >= benchmark['bottom_25']:
        position = "Below industry median"
    else:
        position = "Bottom 25% of industry"
    
    print(f"\n  Industry Position: {position}")
    print(f"\n{'='*60}")